# 13) More on OpenMP and OpenMP Tasks

Last time:
- OpenMP Basics
- `#pragma omp parallel` 
- `#pragma omp simd` 

Today: 
1. [More on OpenMP](#more-on-openmp)  
2. [Memory semantics](#memory-semantics)    
3. [A quick demo on `perf`](#a-quick-demo-on-perf)  
4. [OpenMP Tasks](#openmp-tasks)

## 1. More on OpenMP

What does the compiler do when we add the `#pragma openmp parallel` directive?

```c
static double dot_opt3(size_t n, const double *a, const double *b) {
  double sum = 0;
  omp_set_num_threads(4);
  #pragma omp parallel
  {
    #pragma omp for reduction(+:sum)
    for (size_t i=0; i<n; i++)
      sum += a[i] * b[i];
  }
  return sum;
}
```


In [ ]:
! gcc -Os -march=native -fopenmp ../c_codes/module3-3/dot.c -o dot

In [ ]:
! objdump -d --prefix-addresses -M intel dot | grep dot_opt3

### Anatomy of a parallel region

!["Anatomy of a parallel region"](../img/anatomy_parallel_region.png "The parallelization of dot_opt3")

Where GOMP stands for [GNU Offloading and Multi-Processing Project (GOMP)](https://gcc.gnu.org/projects/gomp/) and is an implementation of OpenMP and OpenACC for GNU compilers.

## 2. Memory semantics

For each variable accessed within the parallel region, we can specify the following data-sharing policies:

- **private**: `private` is the clause that contains the variables that each thread in the OpenMP parallel region will have a copy of. These copies are not initialised upon entering the parallel region.
- **firstprivate**: Like private, but by contrast, `firstprivate` variables are initialised with the value of the original variable upon entering the parallel region.
- **lastprivate**: `lastprivate` is a clause that can be used in a parallelised loop or sections. The `lastprivate` clause shares some of the semantics of the private clause. That is, each thread will have an uninitialised copy of the variables passed as `lastprivate`. However, unlike a private variable, at the end of the parallelised loop or sections, a `lastprivate` variable will take the value of the copy hosted at the thread that executed the last iteration (in the case of a parallelised loop) or section. The "last" iteration or section is the one that would be executed last if they were executed sequentially.
- **shared**: `shared` is the clause that contains the variables shared across the threads belonging to the OpenMP parallel region concerned. Such variables are therefore accessed concurrently, arising potential data-races.

```c
int a=0, b=1, c=2;

#pragma omp parallel private(a) firstprivate(b) shared(c)
{
    int id = omp_get_thread_num();
    a++;
    b++;
    c++;
    printf("[%d] %d %d %d\n", id, a, b, c);
}
printf("END: %d %d %d\n", a, b, c);
```

In [ ]:
! gcc -fopenmp -Wall ../c_codes/module3-3/omp-mem.c -o omp-mem

#### Programming styles

The `private` semantics is actually unnecessary and error-prone. We can just declare those variables at inner-most scope.

```c
int b=1, c=2;

#pragma omp parallel firstprivate(b) shared(c)
{
    int a = 0;
    int id = omp_get_thread_num();
    a++;
    b++;
    c++;
    printf("[%d] %d %d %d\n", id, a, b, c);
}
printf("END: %d %d %d\n", a, b, c); // Error: a not in scope here
```

#### Updating shared variables

We see that the shared variable `c` has lots of opportunities for conflict.

![Updating a shared variable]( ../img/sequence_diagram.png "A sequence diagram")

If we run the above many times, we may sometimes find that multiple processes have the same value of `c`, each thread can observe different increments from others, and the total number of increments may vary.

We can define ordering semantics using:
- [**`atomic`**](https://www.openmp.org/spec-html/5.0/openmpsu95.html#x126-4840002.17.7): The `atomic` construct ensures that a specific storage location is accessed atomically, rather than exposing it to the possibility of multiple, simultaneous reading and writing threads that may result in indeterminate values.
- [**`critical`**](https://www.openmp.org/spec-html/5.0/openmpsu89.html#x120-4470002.17.1): The `critical` construct restricts execution of the associated structured block to a single thread at a time.
- [**`barrier`**](https://www.openmp.org/spec-html/5.0/openmpsu90.html#x121-4550002.17.2): The `barrier` construct specifies an explicit barrier at the point at which the construct appears. The `barrier` construct is a stand-alone directive.


```c
int b=1, c=2;
  
#pragma omp parallel firstprivate(b) shared(c)
{
    int a = 1;
    int id = omp_get_thread_num();
    b++;
    #pragma omp critical
    c++;
    #pragma omp barrier
    printf("[%d] %d %d %d\n", id, a, b, c);
}
printf("END: _ %d %d\n", b, c);
```


## 3. A quick demo on `perf`

Linux [`perf`](https://perfwiki.github.io/main/) is a kernel interrupt-based profiling tool. It uses performance counters and interrupts to diagnose all sorts of bottlenecks.

In [ ]:
! perf stat ../c_codes/module3-3/dot -n 10000 > /dev/null

In [ ]:
! perf record -g ../c_codes/module3-3/dot -n 10000 -r 1000 > /dev/null

In [ ]:
! perf report -M intel

Note how GOMP overhead dominates the cost in this experiment. We need more work (longer arrays, etc.) to justify the overhead of distributing and collecting the parallel work.

We can drill down into particular functions (especially those that we've written, which we have hopefully compiled with `-g` to include debugging information).

![Perf report annotated](../img/perf-report-ann.png  "Perf report annotated")

From this, we see specific instructions, and their corresponding lines of code, that are most frequently being processed when the kernel interrupts to check. In this experiment, we see `*sd` "scalar double" instructions, indicating lack of vectorization.

In contrast, the following annotation shows use of `*pd` "packed double" instructions, indicating that the "hot" loop has been vectorized.

![Perf vectorized report annotated](../img/perf-report-ann-vec.png  "Perf vectorized report annotated")

The reason for vectorization can sometimes be determined by `-fopt-info` `-fopt-info-missed`, and can be encouraged by techniques like manually splitting accumulators, preventing aliasing by using `restrict`, directives like `#pragma omp simd`, and global compiler flags like `-ffast-math` (although, very dangerous to use).

:::{tip}
For more on `perf`, see [Brendan Gregg’s Linux Performance site](http://www.brendangregg.com/linuxperf.html).
:::

## 4. OpenMP Tasks

:::{tip}
See this resource for [OpenMP Task Parallelism](https://fdesprez.github.io/teaching/par-comput/lectures/slides/L4-OpenMP-pt2-2p.pdf).
:::

### Using `#pragma omp task`

Up to now, we’ve been expressing parallelism for iterating over an array.

- The application programmer specifies regions of code to be executed in a task
with the `#pragma omp task` construct
- All tasks can be executed _independently_
- When any thread encounters a task construct, a task is generated
- Tasks are executed **asynchronously** by any thread of the parallel region
- Completion of the tasks can be guaranteed using the `taskwait` synchronization construct

```{literalinclude} ../c_codes/module3-3/task_dep.4.c
:language: c
:linenos: true
```

In [ ]:
! gcc -fopenmp ../c_codes/module3-3/task_dep.4.c -o task_dep.4

In [ ]:
! for i in {1..10}; do ../c_codes/module3-3/task_dep.4; done

- The [**`single`**](https://www.openmp.org/spec-html/5.0/openmpsu38.html) construct specifies that the associated structured block is executed by only one of the threads in the team (not necessarily the master thread), in the context of its implicit task. The other threads in the team, which do not execute the block, wait at an implicit barrier at the end of the `single` construct unless a `nowait` clause is specified.
- The [**`depend`**](https://www.openmp.org/spec-html/5.0/openmpsu99.html#x130-5160002.17.11) clause allows you to provide information on the way a task
will access data
  * It is followed by an access mode that can be `in`, `out` or `inout`. Examples:
  * `depend(in: x, y, z)`: the task will **read** variables `x`, `y` and `z` 
  * `depend(out: res)`: the task will **write** variable `res`; Any previous value of `res` will be ignored and overwritten
  * `depend(inout: k, buffer[0:n])`: the task will both **read and write** the variables `k` and `buffer`; the content of `n` elements of `buffer` starting from index 0 will be used in the read-and-write
- The OpenMP runtime system dynamically decides whether a task is ready for execution or not considering its dependencies (there is no need for further user intervention here).

```{literalinclude} ../c_codes/module3-3/task_dep.4inout.c
:language: c
:linenos: true
```

In [ ]:
! gcc -fopenmp ../c_codes/module3-3/task_dep.4inout.c -o task_dep.4inout

In [ ]:
! for i in {1..10}; do ../c_codes/module3-3/task_dep.4inout; done

In general, creating tasks (even with only one thread) creates an expensive overhead.


### The OpenMP loop scheduler

- When we put together the `#pragma omp parallel` (which spawns a group of threads) and `#pragma omp for` (which divides loop iterations between the spawned threads) constructs, as in `#pragma omp parallel for` we do both things at once.
  * To this, you can optionally add `schedule(static,n)`, where `n` is the chunk size that you want the tasks to be divided into for the threads. (Note: adding `schedule(static,1)` as in `#pragma omp parallel for schedule(static,1)` is equivalent to just `#pragma omp parallel for`)
  * `schedule(dynamic,n)` still tells OpenMP to split task into size chunks, but distribute them to threads dynamically without any specific order.
  * Check other options in this [resource](https://610yilingliu.github.io/2020/07/15/ScheduleinOpenMP/).

### To fork/join or to task?

One of the main issues in High-Performance Computing (HPC) systems is the underutilization of resources. Parallel applications partition and distribute compute and data across processors in the system that work together to solve a given problem. In this operation, processors synchronize and communicate which may lead to some of them spending time idle, waiting for other processors to complete their part. Idle processors mean wasted time and power. This can happen for serial sections of the code, load imbalance, or if you are waiting for synchronization.

These issues are common in bulk synchronous parallel applications, especially those that statically assign work to processors. 

Many codes rely on **bulk synchronous parallelization** constructs to distribute and synchronize work across multiple threads in a system. In this model, multiple threads operate in parallel on different parts of a problem, and perform a global synchronization when the parallel work is completed. 

Fork-join is a similar model where a single thread, sometimes called a master thread, is the application entry point. This forks into multiple threads that concurrently work on different parts of a problem, and then synchronize to join into a single thread when the work in the parallel section is complete (similar to the  worksharing-parallel constructs in OpenMP that distribute the iterations in a loop across multiple threads).

![Bulk synchronous parallelization](../img/bulk-fork-join.png "Bulk synchronous parallelization Vs fork-join model")

**Load imbalance** appears when different threads receive an uneven amount of work to do, or perform the work at different speeds, leading to different amounts of compute time. In this scenario, the faster threads need to wait for lagging threads on global synchronizations, therefore being in an idle state and wasting resources during that time. In the fork-join model, serial sections in between parallel regions become an increasing bottleneck, as parallel regions are shortened with increasing numbers of threads. 

A task is a piece of compute that operates on a piece of data and that may be executed concurrently with other tasks. This parallel programming abstraction is intuitive and allows to specify data-flow dependencies between tasks instead of costly global synchronizations. This mitigates idle time created as a result of load imbalance, given that threads pick up work as they complete, and there is also less time spent on serial sections due to the reduced number of global synchronizations. 

![Tasking](../img/tasking.png "Tasking model")

For tasking to be efficient, it relies on **overdecomposition**, i.e., creating more work units than there are processing units. For many numerical algorithms, there is some overhead to overdecomposition. For example, in array processing of an array size $n$, a halo/fringe/ghost/overlap region might need to be computed as part of each work patch, leading to time models along the lines of

$$
t_{\textrm{tile}} = t_{\textrm{latency}} + \frac{(n+2)^3}{R}
$$

where $R$ is the processing rate. 

:::{tip}
Recommended reading: [Tasking Lives Up to its Promises](https://community.arm.com/arm-research/b/articles/posts/tasking-lives-up-to-its-promises)
:::